In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from utils.train_stft import train_cnn_stft
import matplotlib.pyplot as plt
import numpy as np
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.optuna import OptunaSearch
from ray.air import RunConfig

CONFIG = {
    "dropout": tune.choice([0.1,0.2,0.3,0.4]),
    "f_start": tune.uniform(0, 1000),
    "batch_size": tune.choice([16,32,64]),
    "learning_rate": tune.loguniform(1e-5, 1e-2),
    "input_channels": tune.choice([32,64,128]),
}

reporter = CLIReporter(
        parameter_columns=["input_channels", "dropout", "learning_rate", "batch_size", "f_start"],
        metric_columns=["loss", "accuracy", "training_iteration"])
algo = OptunaSearch()

tuner = tune.Tuner(
        tune.with_resources(
                tune.with_parameters(train_cnn_stft),
                resources={"cpu": 16, "gpu": 2}
        ),
        tune_config=tune.TuneConfig(
                metric='accuracy',
                mode="max",
                search_alg=algo,
                num_samples=5,
        ),
        run_config=RunConfig(
                stop={"training_iteration": 60},
                verbose=2,
        ),
        param_space=CONFIG,
)
results = tuner.fit()

# model = models.resnet18(False)
# model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
# model.fc = torch.nn.Linear(512, 3)

best_result = results.get_best_result("accuracy", "max")
print("Best config is:", results.get_best_result().config)

/root/miniconda3/envs/UGaze/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/UGaze/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /root/miniconda3/envs/UGaze/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warn(f"Failed to load image Python extension: {e}")
2023-04-20 19:54:37,362	WARNING services.py:1780 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 663543808 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in 

/root/miniconda3/envs/UGaze/lib/python3.10/site-packages/optuna/distributions.py:799: FutureWarning: UniformDistribution(high=1000.0, low=0.0) is deprecated and internally converted to FloatDistribution(high=1000.0, log=False, low=0.0, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)
/root/miniconda3/envs/UGaze/lib/python3.10/site-packages/optuna/distributions.py:799: FutureWarning: LogUniformDistribution(high=0.01, low=1e-05) is deprecated and internally converted to FloatDistribution(high=0.01, log=True, low=1e-05, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)


Trial name,accuracy,extrastole_fpr,extrastole_tpr,extrastole_val_fpr,extrastole_val_fpr_denom,extrastole_val_fpr_numer,extrastole_val_tpr,extrastole_val_tpr_denom,extrastole_val_tpr_numer,loss,murmur_fpr,murmur_tpr,murmur_val_fpr,murmur_val_tpr,should_checkpoint,val_accuracy,val_loss
train_cnn_stft_18e76409,0.563959,0,0,0,465,0,0,38,0,0.0301844,0.434548,0.0969697,0.359459,0,True,0.66004,0.0285989


(train_cnn_stft pid=4091823) Number of data per class
(train_cnn_stft pid=4091823) 0    210
(train_cnn_stft pid=4091823) 1    174
(train_cnn_stft pid=4091823) 2    128
(train_cnn_stft pid=4091823) Name: count, dtype: int64
(train_cnn_stft pid=4091823) Number of data per class
(train_cnn_stft pid=4091823) 0    110
(train_cnn_stft pid=4091823) 1     37
(train_cnn_stft pid=4091823) 2     14
(train_cnn_stft pid=4091823) Name: count, dtype: int64
(train_cnn_stft pid=4091823) Number of data per class
(train_cnn_stft pid=4091823) 0    199
(train_cnn_stft pid=4091823) 1    204
(train_cnn_stft pid=4091823) 2    132
(train_cnn_stft pid=4091823) Name: count, dtype: int64
(train_cnn_stft pid=4091823) Number of data per class
(train_cnn_stft pid=4091823) 0    121
(train_cnn_stft pid=4091823) 1     27
(train_cnn_stft pid=4091823) 2     13
(train_cnn_stft pid=4091823) Name: count, dtype: int64


In [ ]:
df = results.get_dataframe()
columns_to_drop = ['time_this_iter_s', 'should_checkpoint', 'done', 'timesteps_total',
       'episodes_total', 'training_iteration', 'trial_id', 'experiment_id',
       'date', 'timestamp', 'time_total_s', 'pid', 'hostname', 'node_ip',
       'time_since_restore', 'timesteps_since_restore',
       'iterations_since_restore', 'warmup_time', 'extrastole_val_tpr_numer', 
       'extrastole_val_tpr_denom', 'extrastole_val_fpr_numer', 'extrastole_val_fpr_denom']
for column in columns_to_drop:
    df = df.drop(columns=column)
path = '/'.join(df['logdir'][0].split('/')[:4])
df.to_csv(f'{path}/results.csv')

In [ ]:
df

,loss,accuracy,murmur_tpr,murmur_fpr,extrastole_tpr,extrastole_fpr,murmur_val_tpr,murmur_val_fpr,val_loss,val_accuracy,extrastole_val_tpr,extrastole_val_fpr,config/batch_size,config/dropout,config/f_start,config/input_channels,config/learning_rate,logdir
0,0.021069,0.889292,0.879093,0.097872,0.710660,0.007735,0.488095,0.339667,0.032046,0.540594,0.16,0.154167,32,0.2,814.416605,128,0.000070,/root/ray_results/train_cnn_stft_2023-04-20_19...
1,0.030080,0.595496,0.480769,0.457105,0.000000,0.000000,0.284404,0.306407,0.027878,0.683761,0.00,0.000000,32,0.3,432.228535,128,0.003189,/root/ray_results/train_cnn_stft_2023-04-20_19...
2,0.027874,0.666375,0.574413,0.303030,0.000000,0.000000,0.644444,0.183673,0.024706,0.785219,0.00,0.000000,32,0.4,2.591720,64,0.005213,/root/ray_results/train_cnn_stft_2023-04-20_19...
3,0.027562,0.687956,0.494475,0.355586,0.000000,0.000000,0.288288,0.212938,0.027318,0.742739,0.00,0.000000,32,0.2,192.009553,32,0.005157,/root/ray_results/train_cnn_stft_2023-04-20_19...
4,0.052627,0.858524,0.913151,0.202721,0.000000,0.000000,0.174419,0.303725,0.060350,0.648276,0.00,0.000000,16,0.3,618.533334,32,0.000012,/root/ray_results/train_cnn_stft_2023-04-20_19...
5,0.027325,0.700823,0.568182,0.352227,0.000000,0.000000,0.095652,0.340720,0.027977,0.663866,0.00,0.000000,32,0.4,746.424666,32,0.000216,/root/ray_results/train_cnn_stft_2023-04-20_19...
6,0.050537,0.836547,0.996825,0.222222,0.000000,0.000000,0.351351,0.305882,0.055245,0.719262,0.00,0.000000,16,0.2,311.457304,64,0.000011,/root/ray_results/train_cnn_stft_2023-04-20_19...
7,0.010762,0.887801,0.975936,0.021362,0.301887,0.002075,0.752577,0.354108,0.016346,0.664444,0.00,0.000000,64,0.3,298.123993,32,0.000220,/root/ray_results/train_cnn_stft_2023-04-20_19...
8,0.026979,0.827216,1.000000,0.103185,0.000000,0.000000,0.903704,0.171687,0.029379,0.841542,0.00,0.000000,32,0.3,36.506448,128,0.000010,/root/ray_results/train_cnn_stft_2023-04-20_19...
9,0.021268,0.875115,0.996951,0.106439,0.000000,0.000000,0.293233,0.472892,0.033146,0.561290,0.00,0.000000,32,0.2,640.170230,32,0.000038,/root/ray_results/train_cnn_stft_2023-04-20_19...
